# Computation with Xarray

- Aggregation: calculation of statistics (e.g. sum) along a dimension of an xarray object can be done by dimension name instead of an integer axis number.
- Arithmetic: arithmetic between xarray objects vectorizes based on dimension names, automatically looping (broadcasting) over each distinct dimension. This eliminates the need to insert dummy dimensions of size one to facilitate broadcasting, a common pattern with NumPy.
- Split-apply-combine: xarray includes N-dimensional grouped operations implementing the split-apply-combine strategy [24].
- Resampling and rolling window operations: Utilizing the efficient resampling methods from pandas and rolling window operations from Bottleneck [15], xarray offers a robust set of resampling and rolling window operations along a single dimension.


### Outline
- Arithmetic
- Aggregation
- Groupby and resample
- Rolling
- Universal Functions

### Tutorial Duriation
10 minutes

### Going Further

- Xarray documentation on its Computation Toolkit: http://xarray.pydata.org/en/latest/computation.html
- Xarray documentation on Groupby: http://xarray.pydata.org/en/latest/groupby.html

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
ds = xr.tutorial.load_dataset('rasm')
da = ds['Tair']

ds

## Aggregation

Xarray supports many of the aggregations methods that numpy has. A partial list includes: all, any, argmax, argmin, max, mean, median, min, prod, sum, std, var.

Whereas the numpy syntax would require scalar axes, xarray can use dimension names:

In [ ]:
da.mean(dim=('x', 'y'))

## Arithmetic

Arithmetic operations with a single DataArray automatically vectorize (like numpy) over all array values:

In [ ]:
# dataarray + scalars
da - 273.15  # (K --> C)

Here we do two computations:
1. Calculate the time-mean
2. Calculate the "anomalies" relative to the time mean

In [ ]:
da_mean = da.mean(dim='time')
da_mean

In [ ]:
# dataarray + dataarray
da - da_mean  

# Notice that this required broadcasting along the time dimension

## Groupby

xarray supports “group by” operations with the same API as pandas to implement the split-apply-combine strategy:

- Split your data into multiple independent groups.
- Apply some function to each group.
- Combine your groups back into a single data object.

Group by operations work on both Dataset and DataArray objects. Most of the examples focus on grouping by a single one-dimensional variable, although support for grouping over a multi-dimensional variable has recently been implemented.

In [ ]:
# Using groupby to calculate a monthly climatology:

da_climatology = da.groupby('time.month').mean('time')

da_climatology

In this case, we provide what we refer to as a virtual variable (`time.month`). Other virtual variables include:
“year”, “month”, “day”, “hour”, “minute”, “second”, “dayofyear”, “week”, “dayofweek”, “weekday” and “quarter”. It is also possible to use another DataArray or pandas object as the grouper.

## Rolling Operations

Xarray objects include a rolling method to support rolling window aggregations:

In [ ]:
roller = da.rolling(time=3)
roller

In [ ]:
roller.mean()

In [ ]:
# we can also provide a custom function 

def sum_minus_2(da, axis):
    return da.sum(axis=axis) - 2

roller.reduce(sum_minus_2)

## Universal Functions

In practice, not all use standard functions/methods from numpy and xarray is not allways possible. Sometimes there is good reason to work with numpy/dask array's directly. 

Xarray's documentation on wrapping custom computations: http://xarray.pydata.org/en/latest/computation.html#wrapping-custom-computation

In [ ]:
da_noise = da + np.random.random(size=da.shape)
da_noise

In [ ]:
# some example legacy code to calculate the spearman correlation coefficient

import bottleneck


def covariance_gufunc(x, y):
    return ((x - x.mean(axis=-1, keepdims=True))
            * (y - y.mean(axis=-1, keepdims=True))).mean(axis=-1)

def correlation_gufunc(x, y):
    return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))

def spearman_correlation_gufunc(x, y):
    x_ranks = bottleneck.rankdata(x, axis=-1)
    y_ranks = bottleneck.rankdata(y, axis=-1)
    return correlation_gufunc(x_ranks, y_ranks)

In [ ]:
# Wrap the "legacy code" with xarray's apply_ufunc. 
def spearman_correlation(x, y, dim):
    return xr.apply_ufunc(
        spearman_correlation_gufunc, x, y,
        input_core_dims=[[dim], [dim]],
        dask='parallelized',
        output_dtypes=[float])

In [ ]:
da_corr = corr = spearman_correlation(da, da_noise, 'time')
da_corr

## Masking

Indexing methods on xarray objects generally return a subset of the original data. However, it is sometimes useful to select an object with the same shape as the original data, but with some elements masked. To do this type of selection in xarray, use where():

In [ ]:
# mask out 1's in the correlation array

da_corr.where(da_corr < 1)

In [ ]:
# xarray also provides a function for where
xr.where(da_corr > 0.996, 0, 1)